In [1]:
import random

from data_generator.csv_data_processor import CSVDataProcessor
from utils.util_funcs import get_row_count, display_df, remove_data, verify_ranking, verify_ranking_counts
from utils.read_write import read_postgres_table
from spark_instance import spark
from pyspark.sql import Window

from pyspark.sql.functions import col, lower, lit, rand, array, floor, concat, when, row_number, concat_ws, date_format, split, size
  

/Users/jacobbickerstaff/Desktop/Development/health-data-etl-kaggle/venv/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
24/02/25 09:40:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# csv_reader = CSVDataProcessor(spark, "data/healthcare_dataset.csv")
# 
# # Read the CSV file
# raw_df = csv_reader.run()


In [3]:
# min_age_days = 1 * 365  # Minimum age in days (18 years)
# max_age_days = 90 * 365  # Maximum age in days (90 years)
# 
# raw_df = (raw_df.withColumn("DOB", expr(f"date_sub(current_date(), CAST(round(rand() * ({max_age_days} - {min_age_days}) + {min_age_days}) AS INT))"))
#         .withColumn("Age", floor(datediff(current_date(), col("DOB")) / 365.25)))




In [4]:
# above is commented out as it's saved in the database:
df = read_postgres_table("dob_age_raw_data")

In [5]:
# df.where(col("Age") < 16).show()

In [6]:
from constants.admission_types_dataset import admission_mapping, AdmissionTypes
flattened = [
    (
        top_level.name,
        sub_level_key.name,
        stay_type,
        sub_level_info.get("conditions",),  
        sub_level_info.get("tests"),  
        random.choice(sub_level_info.get("doctors"))
    )
    for top_level, sub_level_dict in admission_mapping.items()
    for sub_level_key, sub_level_info in sub_level_dict.items()
    for stay_type in (sub_level_info["stay_types"] if isinstance(sub_level_info, dict) else sub_level_info)
]



In [7]:
df.show()

+--------------------+---+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+----------+
|                name|Age|gender|blood_type|medical_condition|date_of_admission|            doctor|            hospital|insurance_provider|    billing_amount|room_number|admission_type|discharge_date| medication|test_results|       DOB|
+--------------------+---+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+----------+
|     Tiffany Ramirez| 39|Female|        O-|         Diabetes|       2022-11-17|    Patrick Parker|    Wallace-Hamilton|          Medicare| 37490.98336352819|        146|      Elective|    2022-12-01|    Aspirin|Inconclusive|1984-09-30|
|         Ruben Burns| 16|  Male|        O+|        

In [8]:
flattened

[('EMERGENCY',
  'INJURY_RTC',
  'Inpatient',
  ['Whiplash',
   'Concussions',
   'Contusions (bruises)',
   'Lacerations',
   'Abrasions',
   'Fractures',
   'Sprains',
   'Strains',
   'Dislocations',
   'Rib Fractures',
   'Internal Bleeding',
   'Pneumothorax (collapsed lung)',
   'Spinal Cord Injuries',
   'Traumatic Brain Injuries (TBIs)',
   'Crush Injuries',
   'Amputations',
   'Psychological Trauma',
   'Burn Injuries',
   'Skull Fractures',
   'Pelvic Fractures',
   'Facial Fractures',
   'Hematomas',
   'Organ Damage',
   'Cervical Spine Injuries',
   'Thoracic Spine Injuries',
   'Lumbar Spine Injuries',
   'Tendon Injuries',
   'Nerve Damage',
   'Eye Injuries',
   'Dental Injuries',
   'Post-Traumatic Arthritis',
   'Compartment Syndrome',
   'Soft Tissue Injuries',
   'Seat Belt Injuries',
   'Airbag Injuries',
   'Lower Extremity Injuries',
   'Upper Extremity Injuries',
   'Drowning or Near-Drowning Incidents',
   'Hypothermia or Frostbite',
   'Heatstroke',
   'Acous

In [9]:
"""
STEP 1 
    From the flattened data list we wish to create a DataFrame. This contains all the possible combinations for the given 
    top level admissions, sub level admissions, stay types and list of tests available from the admission_mapping, stay_type and admission_tests lists or dictionaries in admission_types_test_dataset.py
"""
mapping_df = spark.createDataFrame(flattened, ["top_level_admission", "sub_level_admission", "stay_type", "possible_tests", "conditions", "doctor"])

display_df(mapping_df, 45)

,top_level_admission,sub_level_admission,stay_type,possible_tests,conditions,doctor
0,EMERGENCY,INJURY_RTC,Inpatient,"[Whiplash, Concussions, Contusions (bruises), ...","[X-rays, CT scans, MRI, Ultrasound, Blood tests]",Dr. Cynthia Elliott
1,EMERGENCY,INJURY_RTC,Day Patient,"[Whiplash, Concussions, Contusions (bruises), ...","[X-rays, CT scans, MRI, Ultrasound, Blood tests]",Dr. Kathryn Mercado
2,EMERGENCY,SELF_INFLICTED,Inpatient,[No conditions],[No tests],Dr. Ashley Garrett
3,EMERGENCY,CARDIOLOGY,Inpatient,[No conditions],"[ECG, Echocardiogram, Stress tests, Cardiac ca...",Dr. Joseph Anderson
4,EMERGENCY,CARDIOLOGY,Day Patient,[No conditions],"[ECG, Echocardiogram, Stress tests, Cardiac ca...",Dr. Adriana Montgomery
5,EMERGENCY,NEUROLOGY,Inpatient,"[Migraines, Epilepsy, Parkinson's Disease, Alz...","[MRI or CT scans of the brain, Electroencephal...",Dr. Darrell Hill
6,EMERGENCY,NEUROLOGY,Day Patient,"[Migraines, Epilepsy, Parkinson's Disease, Alz...","[MRI or CT scans of the brain, Electroencephal...",Dr. Robert Mercado
7,EMERGENCY,NEUROLOGY,Outpatient,"[Migraines, Epilepsy, Parkinson's Disease, Alz...","[MRI or CT scans of the brain, Electroencephal...",Dr. Kathryn Hernandez
8,EMERGENCY,GASTROENTEROLOGY,Inpatient,"[Gastro-oesophageal Reflux Disease (GORD), Irr...","[Endoscopy, Colonoscopy, Blood tests, Stool te...",Dr. Shelly Haas
9,EMERGENCY,GASTROENTEROLOGY,Day Patient,"[Gastro-oesophageal Reflux Disease (GORD), Irr...","[Endoscopy, Colonoscopy, Blood tests, Stool te...",Dr. Michelle David


In [10]:
"""
STEP 2 
    Create a list of admission_types randomly assign this to the original patient in the original data set, 
    whilst dropping the original admission_type column. Then join with mapping_df on top_level_admission col to give access to possible 
    conditions, mappings and so on.
"""
admission_type_names = [member.name for member in AdmissionTypes]

print(admission_type_names)

keys_array = array([lit(name) for name in admission_type_names])

# Define constants and conditions
female_only = ["MATERNITY", 'obstetrics']
is_female = lower(col('gender')) == 'female'
is_pediatric = col("Age") < 18
is_geriatric = (col("Age") >= 65) & (col("sub_level_admission") == "GERIATRICS")

df = (df.withColumn("top_level_admission", keys_array[floor(rand() * len(admission_type_names))]).drop("admission_type")
        .withColumn("is_female", is_female)
        .withColumn("is_pediatric", is_pediatric)
        .drop("doctor", "medical_condition", "test_results", "medication")
      )
get_row_count(df, True)
df = df.join(mapping_df, "top_level_admission", "inner")
get_row_count(df, True)
display_df(df, 999)

['EMERGENCY', 'GP_REFERRAL', 'HOSPITAL_REFERRAL', 'SELF_REFERRAL', 'ELECTIVE']
10000
154171


,top_level_admission,name,Age,gender,blood_type,date_of_admission,hospital,insurance_provider,billing_amount,room_number,...,medication,test_results,DOB,is_female,is_pediatric,sub_level_admission,stay_type,possible_tests,conditions,doctor
0,ELECTIVE,Tiffany Ramirez,39,Female,O-,2022-11-17,Wallace-Hamilton,Medicare,37490.983364,146,...,Aspirin,Inconclusive,1984-09-30,True,False,MATERNITY,Inpatient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Ross Greene
1,ELECTIVE,Tiffany Ramirez,39,Female,O-,2022-11-17,Wallace-Hamilton,Medicare,37490.983364,146,...,Aspirin,Inconclusive,1984-09-30,True,False,MATERNITY,Day Patient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Daniel Maxwell
2,ELECTIVE,Tiffany Ramirez,39,Female,O-,2022-11-17,Wallace-Hamilton,Medicare,37490.983364,146,...,Aspirin,Inconclusive,1984-09-30,True,False,MATERNITY,Outpatient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. James Hanson
3,ELECTIVE,Patrick Parker,86,Male,AB+,2020-08-20,Boyd PLC,Aetna,22522.363385,180,...,Aspirin,Abnormal,1938-01-22,False,False,MATERNITY,Inpatient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Ross Greene
4,ELECTIVE,Patrick Parker,86,Male,AB+,2020-08-20,Boyd PLC,Aetna,22522.363385,180,...,Aspirin,Abnormal,1938-01-22,False,False,MATERNITY,Day Patient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Daniel Maxwell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,ELECTIVE,Thomas Berry,30,Female,A-,2021-12-05,Flores Group,Cigna,35807.012158,419,...,Ibuprofen,Inconclusive,1993-12-13,True,False,MATERNITY,Day Patient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Daniel Maxwell
995,ELECTIVE,Thomas Berry,30,Female,A-,2021-12-05,Flores Group,Cigna,35807.012158,419,...,Ibuprofen,Inconclusive,1993-12-13,True,False,MATERNITY,Outpatient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. James Hanson
996,ELECTIVE,Crystal Wright,33,Male,B-,2023-10-11,"Roman, Ruiz and Cox",Aetna,3414.135042,243,...,Aspirin,Normal,1990-10-27,False,False,MATERNITY,Inpatient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Ross Greene
997,ELECTIVE,Crystal Wright,33,Male,B-,2023-10-11,"Roman, Ruiz and Cox",Aetna,3414.135042,243,...,Aspirin,Normal,1990-10-27,False,False,MATERNITY,Day Patient,"[Labour and Delivery, Scheduled Caesarean Sect...","[Ultrasound, Blood tests, Glucose tolerance te...",Dr. Daniel Maxwell


In [11]:
"""
STEP 3 
    Create a rowq ranking using a unique cols, created from stay_name and unique_id. 
    Verification done below. 
    
"""

df =  df.withColumn('stay_name', 
                   when(col('stay_type') == 'out_patient', concat(col('name'), lit('_out_patient')))
                   .when(col('stay_type') == 'inpatient', concat(col('name'), lit('_inpatient')))
                   .when(col('stay_type') == 'day_patient', concat(col('name'), lit('_day_patient')))
                   .otherwise(col('name'))
                  ).withColumn("unique_id", concat_ws("_", "name", date_format("DOB", "yyyyMMdd")))


# Define a window specification that partitions data by 'top_level_admission' (or another unique patient identifier if needed)
windowSpec = Window.partitionBy('stay_name', 'unique_id').orderBy(rand())

# Assign row numbers within each partition in a random order
ranked_df = df.withColumn("row_num", row_number().over(windowSpec))


In [12]:
ranked_df.createGlobalTempView("ranked_df")

unique_dobs_df = spark.sql("""
WITH NameCounts AS (
    SELECT name
    FROM global_temp.ranked_df
)

SELECT DISTINCT r.name, r.unique_id
FROM global_temp.ranked_df r
JOIN NameCounts n ON r.name = n.name
ORDER BY r.name
LIMIT 10
""")

24/02/25 09:41:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [13]:
# COSTLY WAY TO VERIFY THAT THE RANK WORKS BELOW....  due to .collect() and .count() in verify_ranking_counts() function

# Example usage:
unique_names = [row['name'] for row in unique_dobs_df.select("name").collect()]
unique_ids = [row['unique_id'] for row in unique_dobs_df.select("unique_id").collect()]
verify_ranking_counts(df, ranked_df, unique_names, unique_ids)

# this function is faster than above
verify_ranking(df, ranked_df)


In [14]:
"""
Step 4: 
    Sort out geriatrics data and verify that individuals are not geriatrics 
"""
ranked_df = ranked_df.withColumn("is_geriatric", is_geriatric)

not_geriatric_df = ranked_df.where((col('sub_level_admission') == "GERIATRICS") &( col("is_geriatric") == False))

filtered_df = remove_data(ranked_df, not_geriatric_df, (col('sub_level_admission') == "GERIATRICS") ,  ( col("is_geriatric") == False))


In [15]:
not_female_df = filtered_df.where((col("sub_level_admission").isin(female_only)) & (col("is_female") == False))
filtered_df_female = remove_data(filtered_df, not_female_df, (col("gender") == "Male"),  (col("sub_level_admission").isin(female_only)))


In [16]:
# filter pediatric patients who cannot be pregnant (based on legal age in the UK, 16) No assumption made an individual cannot choose to get pregnant before this age. 
# upper age bound defined  here: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4822427/#:~:text=Menopause%20typically%20occurs%20between%2045,reproducing%20many%20years%20before%20menopause.
# between 45 and 55 upper bounding will be 50. # Possiblilty to include outliers in a new func in the future.
under16_female_df = filtered_df_female.where((col("Age") < 16) &
                                             (col("Age") <= 50) &
                                             (col("sub_level_admission") == "MATERNITY")
                                             ).orderBy("Age")

In [17]:
df = remove_data(filtered_df_female, under16_female_df, (col("sub_level_admission") == "MATERNITY"),
                 (col("Age") < 16) & (col("Age") > 50))



In [18]:

split_col = split(df['name'], ' ')

df_renamed = (df.withColumn('name', 
                    when(size(split_col) == 4, concat(split_col.getItem(1).substr(1, 1), lit('. '), split_col.getItem(2).substr(1, 1), lit('. '), split_col.getItem(3)))
                    .when(size(split_col) == 3, concat(split_col.getItem(1).substr(1, 1), lit('. '), split_col.getItem(2)))
                    .otherwise(
                        concat(split_col.getItem(0).substr(1, 1), lit('. '), split_col.getItem(1))
                               )
                    )
      )


In [ ]:
df_renamed.show()

In [26]:
df_renamed.select(["name", "DOB", "Age", "gender", "blood_type", "date_of_admission", "discharge_date", "top_level_admission", "sub_level_admission", "possible_tests", "conditions", "doctor", "hospital", "room_number", "insurance_provider", "billing_amount", "stay_type", "is_female", "is_geriatric", "is_pediatric", "stay_name", "row_num", "unique_id"]).where(col("stay_name") =="Abigail Lamb").show()

+-------+----------+---+------+----------+-----------------+--------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------+-----------+------------------+------------------+-----------+---------+------------+------------+------------+-------+--------------------+
|   name|       DOB|Age|gender|blood_type|date_of_admission|discharge_date|top_level_admission| sub_level_admission|      possible_tests|          conditions|              doctor|hospital|room_number|insurance_provider|    billing_amount|  stay_type|is_female|is_geriatric|is_pediatric|   stay_name|row_num|           unique_id|
+-------+----------+---+------+----------+-----------------+--------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------+-----------+------------------+------------------+-----------+---------+------------+------------+------------+-------+--------------------+
|A. Lamb|2005

In [27]:
df_new = df_renamed.drop("row_num")

windowSpec = Window.partitionBy('stay_name', 'unique_id').orderBy(rand())

# Assign row numbers within each partition in a random order
df_new_part = df_new.withColumn("row_num", row_number().over(windowSpec))

df_new_part.select(["name", "DOB", "Age", "gender", "blood_type", "date_of_admission", "discharge_date", "top_level_admission", "sub_level_admission", "possible_tests", "conditions", "doctor", "hospital", "room_number", "insurance_provider", "billing_amount", "stay_type", "is_female", "is_geriatric", "is_pediatric", "stay_name", "row_num", "unique_id"]).where(col("stay_name") =="Abigail Lamb").show() 

+-------+----------+---+------+----------+-----------------+--------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------+-----------+------------------+------------------+-----------+---------+------------+------------+------------+-------+--------------------+
|   name|       DOB|Age|gender|blood_type|date_of_admission|discharge_date|top_level_admission| sub_level_admission|      possible_tests|          conditions|              doctor|hospital|room_number|insurance_provider|    billing_amount|  stay_type|is_female|is_geriatric|is_pediatric|   stay_name|row_num|           unique_id|
+-------+----------+---+------+----------+-----------------+--------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------+-----------+------------------+------------------+-----------+---------+------------+------------+------------+-------+--------------------+
|A. Lamb|2005

In [20]:
["name", "DOB", "Age", "gender", "blood_type", "date_of_admission", "discharge_date", "top_level_admission", "sub_level_admission", "possible_tests", "conditions", "doctor", "hospital", "room_number", "insurance_provider", "billing_amount", "stay_type", "is_female", "is_geriatric" "is_pediatric", "stay_name", "row_num", "unique_id"]

['sub_level_admission',
 'DOB',
 'top_level_admission',
 'name',
 'Age',
 'gender',
 'blood_type',
 'date_of_admission',
 'hospital',
 'insurance_provider',
 'billing_amount',
 'room_number',
 'discharge_date',
 'medication',
 'test_results',
 'is_female',
 'is_pediatric',
 'stay_type',
 'possible_tests',
 'conditions',
 'doctor',
 'stay_name',
 'unique_id',
 'row_num',
 'is_geriatric']

In [18]:
# TODO:  
# medical condition to be  chosen 
# tests to be chosen
# admission date to be checked again dob,  
# TODO filter on is pediatric, geriatric and is_female to be done here and same people with dob? needs  to be considered 
# drop stay_name and unique_id 


In [19]:
# Filter to keep only the top-ranked row within each partition
ranked_df = filtered_df_female

In [20]:
df_renamed.where(col("name") == "D. Mccoy").show(n=8000)

+-------------------+----------+-------------------+--------+---+------+----------+-----------------+-----------------+-------------+------------------+------------------+-----------+--------------+----------+------------+---------+------------+-----------+----------------------------+--------------------+------------------+------------+--------------------+-------+------------+
|sub_level_admission|       DOB|top_level_admission|    name|Age|gender|blood_type|medical_condition|date_of_admission|     hospital|insurance_provider|    billing_amount|room_number|discharge_date|medication|test_results|is_female|is_pediatric|  stay_type|possible_tests_or_procedures|          conditions|            doctor|   stay_name|           unique_id|row_num|is_geriatric|
+-------------------+----------+-------------------+--------+---+------+----------+-----------------+-----------------+-------------+------------------+------------------+-----------+--------------+----------+------------+---------+----

In [21]:
from pyspark.sql.types import StringType
get_row_count(df, True)
df = df_renamed.select([col(c).cast(StringType()).alias(c) for c in df.columns])
df.repartition(1000).write.csv('./temp_data/renamed/renamed.csv', mode = 'overwrite', header=True)

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/Users/jacobbickerstaff/Desktop/Development/health-data-etl-kaggle/venv/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobbickerstaff/Desktop/Development/health-data-etl-kaggle/venv/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# TODO initialise first name to make data more realistic for name columns i.e. Daniel Mccoy is seen a Male name but here its Female

In [ ]:
spark.stop()

In [ ]:
dict_ = admission_mapping.get("emergency")